In [3]:
// Cell 1: Imports and dependencies
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.3")
@file:DependsOn("com.opencsv:opencsv:5.5")

import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.Solver
import java.time.LocalDateTime
import java.time.LocalTime
import java.time.DayOfWeek
import java.time.Duration
import kotlin.math.sqrt
import com.opencsv.CSVWriter
import java.io.FileWriter

In [4]:
// Cell 2: Data classes
data class Location(
    val name: String,
    val latitude: Double,
    val longitude: Double
) {
    fun distanceTo(other: Location): Double {
        val latDiff = latitude - other.latitude
        val lonDiff = longitude - other.longitude
        return sqrt(latDiff * latDiff + lonDiff * lonDiff)
    }
}

data class Skill(val name: String)

data class Shift(
    val name: String,
    val startTime: LocalTime,
    val endTime: LocalTime
)

data class Agent(
    @PlanningId val id: Long,
    val name: String,
    val skills: List<Skill>,
    val homeLocation: Location
)

data class Meeting(
    @PlanningId val id: Long,
    val client: String,
    val location: Location,
    val requiredSkills: List<Skill>,
    val startDateTime: LocalDateTime,
    val endDateTime: LocalDateTime,
    val requiredAgents: Int,
    val shift: Shift
) {
    fun duration(): Duration = Duration.between(startDateTime, endDateTime)
}

@PlanningEntity
data class AgentAssignment(
    @PlanningId val id: Long,
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"])
    var agent: Agent? = null,
    val meeting: Meeting
)

@PlanningSolution
data class Schedule(
    @ValueRangeProvider(id = "agentRange")
    @ProblemFactCollectionProperty
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    val meetings: List<Meeting>,

    @PlanningEntityCollectionProperty
    val assignments: List<AgentAssignment>,

    @ProblemFactCollectionProperty
    val skills: List<Skill>,

    @ProblemFactCollectionProperty
    val shifts: List<Shift>,

    @PlanningScore
    var score: HardMediumSoftLongScore? = null
) {
    // No-arg constructor for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList(), emptyList(), emptyList())
}

In [5]:
// Cell 3: Data generation
val numberOfAgents = 10
val numberOfMeetings = numberOfAgents * 5 * 7 // Approximately 5 meetings per agent per day for a week

val skills = listOf(
    Skill("Fire Certification"),
    Skill("Bodyguard Certification"),
    Skill("First Aid Certification"),
    Skill("Fighting Certification")
)

val shifts = listOf(
    Shift("Morning", LocalTime.of(8, 0), LocalTime.of(16, 0)),
    Shift("Afternoon", LocalTime.of(11, 0), LocalTime.of(19, 0)),
    Shift("Night", LocalTime.of(20, 0), LocalTime.of(4, 0)) // Ends at 4 am next day
)

fun generateLocations(count: Int): List<Location> {
    val genevaLatitude = 46.2044
    val genevaLongitude = 6.1432
    return (1..count).map { id ->
        Location(
            name = "Location_$id",
            latitude = genevaLatitude + (0.02 * (Math.random() - 0.5)),
            longitude = genevaLongitude + (0.02 * (Math.random() - 0.5))
        )
    }
}

fun generateAgents(count: Int, skills: List<Skill>, homeLocation: Location): List<Agent> {
    return (1..count).map { id ->
        Agent(
            id = id.toLong(),
            name = "Agent_$id",
            skills = skills.shuffled().take((1..skills.size).random()),
            homeLocation = homeLocation
        )
    }
}

fun generateMeetings(
    count: Int,
    skills: List<Skill>,
    locations: List<Location>,
    shifts: List<Shift>
): List<Meeting> {
    val startDate = LocalDateTime.now().withHour(0).withMinute(0).withSecond(0).withNano(0)
    return (1..count).map { id ->
        val requiredSkillCount = (1..3).random()
        val requiredSkills = skills.shuffled().take(requiredSkillCount)
        val location = locations.random()
        val shift = shifts.random()
        val dayOffset = (0..6).random() // 0-6 for a week
        val startDateTime = startDate.plusDays(dayOffset.toLong())
            .withHour(shift.startTime.hour)
            .withMinute(shift.startTime.minute)
            .plusHours((0..7).random().toLong())
        val duration = Duration.ofHours((1..3).random().toLong())
        val endDateTime = startDateTime.plus(duration)
        val isNightShift = shift.name == "Night"
        val requiredAgents = if (isNightShift) (1..2).random() else 1

        val meetingSkills = if (isNightShift) {
            val fightingSkill = skills.find { it.name == "Fighting Certification" }!!
            if (requiredSkills.contains(fightingSkill)) requiredSkills else requiredSkills + fightingSkill
        } else {
            requiredSkills
        }

        Meeting(
            id = id.toLong(),
            client = "Client_$id",
            location = location,
            requiredSkills = meetingSkills,
            startDateTime = startDateTime,
            endDateTime = endDateTime,
            requiredAgents = requiredAgents,
            shift = shift
        )
    }
}

// Generate data
val homeLocation = Location("Home_Geneva", 46.2044, 6.1432)
val meetingLocations = generateLocations(20)
val agents = generateAgents(numberOfAgents, skills, homeLocation)
val meetings = generateMeetings(numberOfMeetings, skills, meetingLocations, shifts)

// Generate AgentAssignments
var assignmentId = 1L
val assignments = meetings.flatMap { meeting ->
    (1..meeting.requiredAgents).map {
        AgentAssignment(
            id = assignmentId++,
            agent = null,
            meeting = meeting
        )
    }
}

// Create the initial schedule
val initialSchedule = Schedule(agents, meetings, assignments, skills, shifts)

In [6]:
// Cell 4: Constraint provider
import ai.timefold.solver.core.api.score.buildin.hardmediumsoftlong.HardMediumSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import ai.timefold.solver.core.api.score.stream.bi.BiConstraintCollector
import ai.timefold.solver.core.api.score.stream.bi.BiConstraintStream
import java.time.Duration



In [7]:
// Cell 8: ConstraintProvider Implementation

class SecurityConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            agentConflict(constraintFactory),
            skillRequirements(constraintFactory),
            requiredAgentsPerMeeting(constraintFactory),
            avoidConsecutiveNightShifts(constraintFactory),
            respectBreakTime(constraintFactory),
            minimizeTravelTime(constraintFactory),
            balanceWorkload(constraintFactory)
        )
    }

    fun agentConflict(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.overlapping(
                    { it.meeting.startDateTime },
                    { it.meeting.endDateTime }
                )
            )
            .penalizeLong("Agent conflict", HardMediumSoftLongScore.ONE_HARD) { assignment1: AgentAssignment, assignment2: AgentAssignment ->
                val overlapStart = maxOf(assignment1.meeting.startDateTime, assignment2.meeting.startDateTime)
                val overlapEnd = minOf(assignment1.meeting.endDateTime, assignment2.meeting.endDateTime)
                Duration.between(overlapStart, overlapEnd).toMinutes()
            }
    }

    fun skillRequirements(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(AgentAssignment::class.java)
            .filter { assignment ->
                assignment.agent?.skills?.containsAll(assignment.meeting.requiredSkills) != true
            }
            .penalizeLong("Skill requirements", HardMediumSoftLongScore.ONE_HARD) { assignment: AgentAssignment ->
                assignment.meeting.requiredSkills.size.toLong()
            }
    }

    fun requiredAgentsPerMeeting(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Meeting::class.java)
            .join(AgentAssignment::class.java,
                Joiners.equal(Meeting::id, AgentAssignment::meeting, Meeting::id)
            )
            .groupBy({ meeting, _ -> meeting },
                countAgents())
            .filter { meeting, count -> count != meeting.requiredAgents }
            .penalizeLong("Required agents per meeting", HardMediumSoftLongScore.ONE_HARD) { meeting: Meeting, count: Int ->
                Math.abs(count - meeting.requiredAgents).toLong()
            }
    }

    private fun countAgents(): BiConstraintCollector<Meeting, AgentAssignment, ?, Int> {
        return BiConstraintCollector.of(
            { 0 },
            { _, _: AgentAssignment, count: Int -> count + 1 },
            { count -> count }
        )
    }

    fun avoidConsecutiveNightShifts(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.equal(
                    { it.meeting.shift.name },
                    { it.meeting.shift.name }
                ),
                Joiners.equal(
                    { it.meeting.startDateTime.toLocalDate() },
                    { it.meeting.startDateTime.toLocalDate().plusDays(1) }
                )
            )
            .penalizeLong("Avoid consecutive night shifts", HardMediumSoftLongScore.ONE_MEDIUM) { _, _ -> 1L }
    }

    fun respectBreakTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.lessThan({ it.meeting.endDateTime },
                    { it.meeting.startDateTime })
            )
            .filter { assignment1: AgentAssignment, assignment2: AgentAssignment ->
                Duration.between(assignment1.meeting.endDateTime, assignment2.meeting.startDateTime).toHours() < 8
            }
            .penalizeLong("Respect break time", HardMediumSoftLongScore.ONE_SOFT) { assignment1: AgentAssignment, assignment2: AgentAssignment ->
                val gap = Duration.between(assignment1.meeting.endDateTime, assignment2.meeting.startDateTime).toHours()
                (8 - gap).toLong()
            }
    }

    fun minimizeTravelTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEachUniquePair(AgentAssignment::class.java,
                Joiners.equal(AgentAssignment::agent),
                Joiners.lessThan({ it.meeting.endDateTime },
                    { it.meeting.startDateTime })
            )
            .penalizeLong("Minimize travel time", HardMediumSoftLongScore.ONE_SOFT) { assignment1: AgentAssignment, assignment2: AgentAssignment ->
                (assignment1.meeting.location.distanceTo(assignment2.meeting.location) * 1000).toLong()
            }
    }

    fun balanceWorkload(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Agent::class.java)
            .join(AgentAssignment::class.java,
                Joiners.equal({ agent: Agent -> agent }, AgentAssignment::agent)
            )
            .groupBy({ agent, _ -> agent },
                sumDuration())
            .filter { _, totalHours -> totalHours > 40 }
            .penalizeLong("Balance workload", HardMediumSoftLongScore.ONE_SOFT) { _, totalHours: Int ->
                (totalHours - 40).toLong()
            }
    }

    private fun sumDuration(): BiConstraintCollector<Agent, AgentAssignment, ?, Int> {
        return BiConstraintCollector.of(
            { 0 },
            { _, assignment: AgentAssignment, sum: Int -> sum + assignment.meeting.duration().toHours().toInt() },
            { sum -> sum }
        )
    }
}


Line_7.jupyter.kts (57:80 - 81) Type expected
Line_7.jupyter.kts (123:78 - 79) Type expected